In [ ]:
!pip install contractions -q
!pip install textacy -q #based on spacy

import contractions
import nltk
import re
import string
nltk.download('punkt') #Dictionary for punctuation
nltk.download('averaged_perceptron_tagger') #for tokenization
nltk.download('stopwords')
nltk.download('stem_porter')
nltk.download('wordnet')

from textacy import preprocessing
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.7 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Error loading stem_porter: Package 'stem_porter' not found
[nltk_data]     in index
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
import pandas as pd
import re
import nltk
from textacy import preprocessing
import contractions

# Ensure necessary NLTK data is downloaded
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def f_preprocess_data(preprocess_text):
    """
    Preprocesses the given text by applying cleaning, normalization, and tokenization steps,
    including removal of URLs, specific sequences like '&amp;#x200B;', and unusual characters.

    Parameters:
    preprocess_text (str): The text to preprocess.

    Returns:
    str: The preprocessed text.
    """
    if pd.isnull(preprocess_text):
        return ""

    # Remove URLs
    preprocess_text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', preprocess_text)

    # Remove specific sequences like '&amp;#x200B;'
    preprocess_text = preprocess_text.replace('&amp;#x200B;', '')

    # Handling contractions and specific patterns before tokenization
    preprocess_text = contractions.fix(preprocess_text, slang=False)
    preprocess_text = preprocessing.replace.hashtags(preprocess_text, repl="_hashtag_")
    preprocess_text = preprocessing.replace.user_handles(preprocess_text, repl="_user_")
    preprocess_text = preprocessing.replace.emojis(preprocess_text, repl="_emoji_")
    preprocess_text = preprocessing.replace.urls(preprocess_text, repl="_url_")

    # Replace or remove unusual characters (keeping only letters, numbers, basic punctuation, and whitespace)
    preprocess_text = re.sub(r'[^a-zA-Z0-9\s.,!?-]', '', preprocess_text)

    # Tokenization
    tokens = word_tokenize(preprocess_text)

    # Discard texts shorter than 5 words
    if len(tokens) < 5:
        return "Insufficient context"  # or return "" to indicate no useful content

    tokens = [token for token in tokens if token not in stop_words]
    tokens = [stemmer.stem(token) for token in tokens]
    tokens = [lemmatizer.lemmatize(token, pos='v') for token in tokens]
    tokens = [token.lower() for token in tokens]
    tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]
    tokens = [token for token in tokens if token != '']

    return ' '.join(tokens)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
import pandas as pd

fileIn = "/content/2 Anxiety_April2019 - MSDS-NLP 2024.xlsx"

df = pd.read_excel( fileIn, index_col=None )

df.keys()

Index(['subreddit', 'id', 'title', 'date', 'selftext'], dtype='object')

In [ ]:
df["preprocess_text"] = df["selftext"].apply(f_preprocess_data)

In [ ]:
# Save the concatenated DataFrame back to the Excel file
fileOut = "Preprocessed_Anxiety_Data_23MSDS15.xlsx"
df.to_excel(fileOut, index=False)